In [606]:
import pandas as pd
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pycountry

df = pd.read_csv('../../data/merged_data_sentiment.csv')
df.head()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/968782246.py:7: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



,ID,Content,User,Date,Location,Reactions,N_Children,Post Title,Platform,meta,lang,Subreddit,Unnamed: 0,Sentiment,Country
0,UgyDawiDJRU5j8e9g6x4AaABAg,i love my mum~!TAT,carol lui,2013-04-30,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCgxe0Tv...",en,NaN,NaN,"0.9998131394386292,0.9847044944763184,0.992429...",NaN
1,Ugx5UKqZcGpX0Qi8art4AaABAg,I was in tears until the lady gave her mother ...,C Ladner,2013-05-13,NaN,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UC98Fh4X...",en,NaN,NaN,"-0.8607058525085449,-0.4791601002216339,0,0.0,...",NaN
2,UgybIB4C2ppmGqU40WV4AaABAg,"I like her voice!! So deep, i thought the voic...",joseph jr.,2013-05-24,PH,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCUe_b0X...",en,NaN,NaN,"0.9991986155509949,0.9696046113967896,0.984618...",PHL
3,Ugyi1lQIQnmZ2jVwk4B4AaABAg,A verve and flow equal to Maria Carrey's debut...,medusanerve,2013-05-28,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCILrk7Q...",en,NaN,NaN,"0.9990953207015991,0.901877224445343,0.9919101...",NaN
4,Ugw2iIVWPilqFB7RMEl4AaABAg,great voice:D\n,ramon goedendorp,2013-06-08,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCNs25ls...",en,NaN,NaN,"0.9998722076416016,0.9386110305786133,0.989294...",NaN


In [607]:
# only looking at rows with country information
df = df[df['Country'].notna()].reset_index(drop=True)
df.shape

(9804, 15)

In [608]:
# get mean sentiment for each comment
def avg_sent(df):
  for i, row in df.iterrows():
    sent = row['Sentiment'].split(',')
    sent = [float(s) for s in sent]
    df.at[i, 'Avg_Sentiment'] = sum(sent) / len(sent)
avg_sent(df)

In [609]:
# get average sentiment of each country
df = df.groupby('Country').mean().reset_index()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/985466259.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [610]:
# plot on a world map (green for positive, red for negative)
fig = go.Figure(data=go.Choropleth(
    locations = df['Country'],
    z = df['Avg_Sentiment'],
    text = df['Country'],
    colorscale = 'RdYlGn',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'Sentiment',
))

fig.update_layout(
    title_text='Sentiment of Comments by Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
)

fig.show()

In [611]:
# print value counts of countries
df = pd.read_csv('../../data/merged_data_sentiment.csv')
df = df[df['Country'].notna()].reset_index(drop=True)
df = df.groupby('Country').count().reset_index()
df = df[['Country', 'ID']]
df.columns = ['Country', 'Count']
df.sort_values(by='Count', ascending=False).head(n=10)

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/3936816153.py:2: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



,Country,Count
129,USA,4168
58,IND,988
46,GBR,769
22,CAN,480
8,AUS,268
44,FRA,264
33,DEU,216
91,NGA,151
101,PHL,124
97,PAK,111


In [612]:
# print percentage of posts from top 10 countries
df.sort_values(by='Count', ascending=False).head(n=10)['Count'].sum() / df['Count'].sum()


0.7980311209907908

In [613]:
# pie chart of how many posts are from each country
df = pd.read_csv('../../data/merged_data_sentiment.csv')
df = df[df['Country'].notna()].reset_index(drop=True)
df = df.groupby('Country').count().reset_index()
df = df[['Country', 'ID']]
df.columns = ['Country', 'Count']

# combine countries with less than 100 posts into 'Other'
df = df.sort_values(by=['Count'], ascending=False)
df = df.reset_index(drop=True)
df.loc[10] = ['Other', df['Count'][10:].sum()]
df = df[:11]

# convert the countries from alpha_3 to name
for i, row in df.iterrows():
  if row['Country'] == 'Other':
    continue
  country = pycountry.countries.get(alpha_3=row['Country'])
  df.at[i, 'Country'] = country.name

# plot pie chart (with Other being the last slice in gray)
colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)', 'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)', 'rgb(127, 127, 127)', 'rgb(188, 189, 34)', 'rgb(23, 190, 207)', 'rgb(211, 211, 211)']
fig = go.Figure(data=[go.Pie(labels=df['Country'], values=df['Count'], hole=0.5, marker=dict(colors=colors))])
fig.update_traces(sort=False, textfont_size=14)
fig.update_traces(textinfo='label+percent', marker=dict(colors=colors + ['rgb(211, 211, 211)']))
fig.update_layout(showlegend=False)
fig.update_layout(title_text='Percentage of Posts by Country')
fig.show()


/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/3977885527.py:2: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



In [614]:
df = pd.read_csv('../../data/merged_data_sentiment.csv')
df = df[df['Country'].notna()].reset_index(drop=True)
df = df.groupby('Country').count().reset_index()
df = df[['Country', 'ID']]
df.columns = ['Country', 'Count']

# take log of volume to make it easier to visualize
df['Log_Count'] = df['Count'].apply(lambda x: 0 if x == 0 else math.log(x, 10))

# world map visualizing volume in each country
fig = go.Figure(data=go.Choropleth(
    locations = df['Country'],
    z = df['Log_Count'],
    text = df['Country'],
    colorscale = 'Blues',
    autocolorscale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'Count',
    zmin=0,
    zmax=3.6,
    colorbar=dict(
        tickvals=[0, 1, 2, 3],
        ticktext=['0', '10', '100', '1000']
    )
))

fig.update_layout(
    title_text='Number of Posts by Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
)

fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/108333972.py:1: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



In [615]:
# countries with more than 100 comments
top_countries = ['USA', 'IND', 'GBR', 'CAN', 'AUS', 'FRA', 'DEU', 'NGA', 'PHL', 'PAK']

In [620]:
# average sentiment in each of the top 10 countries
df = pd.read_csv('../../data/merged_data_sentiment.csv')
df = df[df['Country'].notna()].reset_index(drop=True)
avg_sent(df)
df = df.groupby('Country').mean().reset_index()
df = df[df['Country'].isin(top_countries)].reset_index(drop=True)
# convert the countries from alpha_3 to name
for i, row in df.iterrows():
  country = pycountry.countries.get(alpha_3=row['Country'])
  df.at[i, 'Country'] = country.name
df = df.sort_values(by='Avg_Sentiment', ascending=False)
df = df.reset_index(drop=True)
df = df[['Country', 'Avg_Sentiment']]
df.columns = ['Country', 'Sentiment']
# print without index
print(df.to_string(index=False))


/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/4131013005.py:2: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



       Country  Sentiment
         India   0.252345
      Pakistan   0.206966
   Philippines   0.197035
       Germany   0.168736
        France   0.134678
       Nigeria   0.100213
     Australia   0.042969
United Kingdom   0.036964
        Canada   0.018520
 United States   0.002273


/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/4131013005.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [617]:
# line chart of average sentiment in each of the top 10 countries over time
df = pd.read_csv('../../data/merged_data_sentiment.csv')
df = df[df['Country'].notna()].reset_index(drop=True)

# only get 2019 onward
df = df[df['Date'] >= '2019-01-01'].reset_index(drop=True)

avg_sent(df)

# get year from date
df['Year'] = df['Date'].apply(lambda x: x.split('-')[0])

# get average sentiment of each country
df = df.groupby(['Country', 'Year']).mean().reset_index()

# sort by year
df = df.sort_values(by='Year')

# get top 10 countries (ie all those with >100 posts)
df = df[df['Country'].isin(top_countries)]

# plot (all countries on same plot)
fig = go.Figure()
for country in top_countries:
  temp = df[df['Country'] == country]
  country_name = pycountry.countries.get(alpha_3=country).name
  fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Avg_Sentiment'], name=country_name))

fig.update_layout(title_text='Average Sentiment of Comments by Country Over Time (2019 onward)')
fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/470512505.py:2: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/470512505.py:14: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [618]:
# compare sentiment by income level
df = pd.read_csv('../../data/merged_data_sentiment.csv')
# only posts with Country
df = df[df['Country'].notna()].reset_index(drop=True)
avg_sent(df)
df = df[['Country', 'Avg_Sentiment']]
df['Count'] = 1
df.columns = ['Country', 'Sentiment', 'Count']
df = df.groupby('Country').agg({'Sentiment': 'mean', 'Count': 'sum'}).reset_index()

# income dataset from the World Bank
income_df = pd.read_excel('https://datacatalogfiles.worldbank.org/ddh-published/0037712/DR0090755/CLASS.xlsx')
# merge "Income group" into df (Code in income_df is alpha_3)
df = df.merge(income_df[['Code', 'Income group']], left_on='Country', right_on='Code')

# group by income level
df = df.groupby('Income group').agg({'Sentiment': 'mean', 'Count': 'sum'}).reset_index()
print(df.to_string(index=False))

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/2650856061.py:2: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



       Income group  Sentiment  Count
        High income   0.070132   7469
         Low income   0.040576     32
Lower middle income   0.156520   1813
Upper middle income   0.083139    468


In [619]:
# plot line chart of sentiment by income level
df = pd.read_csv('../../data/merged_data_sentiment.csv')
# only posts with Country
df = df[df['Country'].notna()].reset_index(drop=True)
# only 2019 onward
df = df[df['Date'] >= '2019-01-01'].reset_index(drop=True)
avg_sent(df)
# only look at year
df['Year'] = df['Date'].apply(lambda x: x.split('-')[0])
df = df[['Country', 'Year', 'Avg_Sentiment']]
df.columns = ['Country', 'Year', 'Sentiment']
df = df.groupby(['Country', 'Year']).mean().reset_index()

# income dataset from the World Bank
income_df = pd.read_excel('https://datacatalogfiles.worldbank.org/ddh-published/0037712/DR0090755/CLASS.xlsx')
# merge "Income group" into df (Code in income_df is alpha_3)
df = df.merge(income_df[['Code', 'Income group']], left_on='Country', right_on='Code')

# group by income level
df = df.groupby(['Income group', 'Year']).mean().reset_index()

# plot line chart of sentiment by income level
fig = go.Figure()
for income in ['High income', 'Upper middle income', 'Lower middle income']:
  temp = df[df['Income group'] == income]
  fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Sentiment'], name=income))

fig.update_layout(title_text='Average Sentiment of Comments by Income Level Over Time (2019 onward)')
fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/1291258584.py:2: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_77727/1291258584.py:20: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

